In [1]:
#Imports neccessary packages
import pandas as pd
import numpy as np
import econtools.metrics as mt

#Reads *.dta files into pandas
crime = pd.read_stata("C:\\Users\\vboxuser\\Desktop\\Replication Papers\\CanAlcoholProhibitionReduceViolenceAgainstWomen\\113426-V1\\AERPP-Replication-Data\\LeeLucaOwensSharma_AER_PP_CrimeData.dta")
nfhs = pd.read_stata("C:\\Users\\vboxuser\\Desktop\\Replication Papers\\CanAlcoholProhibitionReduceViolenceAgainstWomen\\113426-V1\\AERPP-Replication-Data\\LeeLucaOwensSharma_AER_PP_NFHSData.dta")

In [2]:
#Converts strings to numerics
crime.replace(('Yes', 'No'), (1, 0), inplace=True)
nfhs.replace(('Yes', 'No'), (1, 0), inplace=True)

#This starts the years at zero instead of i.e. 1980
crime["year"] = pd.to_numeric(crime['year'],downcast="integer")
nfhs["year"] = pd.to_numeric(nfhs['year'],downcast="integer")

#Factorizes qualitative data into quantitative values
nfhs["State"] = pd.factorize(nfhs["State"])[0]
nfhs["religion"] = pd.factorize(nfhs["religion"])[0]
nfhs["husb_age_cat"] = pd.factorize(nfhs["husb_age_cat"])[0]
nfhs["rep_age_cat"] = pd.factorize(nfhs["rep_age_cat"])[0]
nfhs["agegap_cat"] = pd.factorize(nfhs["agegap_cat"])[0]
nfhs["educgap_cat"] = pd.factorize(nfhs["educgap_cat"])[0]

crime["State"] = pd.factorize(crime["State"])[0]

#educgap_cat dummies
ieduccat = pd.get_dummies(nfhs['educgap_cat'],prefix="educcat",dtype=float)
for i in range(0,len(ieduccat.columns)):
    string = ieduccat.columns[i]
    nfhs[string] = ieduccat[string]

#agegap_cat dummies
iagecat = pd.get_dummies(nfhs['agegap_cat'],prefix="agecat",dtype=float)
for i in range(0,len(iagecat.columns)):
    string = iagecat.columns[i]
    nfhs[string] = iagecat[string]

#rep_age_cat dummies
irepcat = pd.get_dummies(nfhs['rep_age_cat'],prefix="repcat",dtype=float)
for i in range(0,len(irepcat.columns)):
    string = irepcat.columns[i]
    nfhs[string] = irepcat[string]
    
#husb_age_cat dummies
ihusbcat = pd.get_dummies(nfhs['husb_age_cat'],prefix="husbcat",dtype=float)
for i in range(0,len(ihusbcat.columns)):
    string = ihusbcat.columns[i]
    nfhs[string] = ihusbcat[string]

#Creates religion dummies
ireligion = pd.get_dummies(nfhs['religion'],prefix="religion",dtype=float)
for i in range(0,len(ireligion.columns)):
    string = ireligion.columns[i]
    nfhs[string] = ireligion[string]

#Creates year dummies
iyear = pd.get_dummies(nfhs['year'],prefix="year",dtype=float)
for i in range(0,len(iyear.columns)):
    string = iyear.columns[i]
    nfhs[string] = iyear[string]
    
iyear = pd.get_dummies(crime['year'],prefix="year",dtype=float)
for i in range(0,len(iyear.columns)):
    string = iyear.columns[i]
    crime[string] = iyear[string]
    
#Creates state dummies
istate = pd.get_dummies(nfhs['State'],prefix="state",dtype=float)
for i in range(0,len(istate.columns)):
    string = istate.columns[i]
    nfhs[string] = istate[string]
    
istate = pd.get_dummies(crime['State'],prefix="state",dtype=float)
for i in range(0,len(istate.columns)):
    string = istate.columns[i]
    crime[string] = istate[string]

C:\Users\vboxuser\AppData\Local\Temp\ipykernel_6964\2599608183.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nfhs[string] = istate[string]
C:\Users\vboxuser\AppData\Local\Temp\ipykernel_6964\2599608183.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nfhs[string] = istate[string]
C:\Users\vboxuser\AppData\Local\Temp\ipykernel_6964\2599608183.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

In [3]:
#Converts any categorical data types to float64 for use in regession-this should not change the value of any data just the way it is stored
for col in list(nfhs.select_dtypes(include=["category"]).columns):
    nfhs[col] = nfhs[col].astype('float64')

In [4]:
#The exogenous variables to be used for predictions
x1 = ["prohib", "cons"]
x2 = ["husb_age", "husb_educ", "husb_wc", "urban", "hhsize", "cons"]
x3 = ["husb_age", "husb_educ", "husb_wc", "urban", "hhsize", "children", "rep_educ", "rep_age", "rep_wc", "r_educ", "r_age", "b_unfaithful", "ownmoney", "cons"]
x4 = ["husb_age", "husb_educ", "husb_wc", "urban", "hhsize", "children", "rep_educ", "rep_age", "rep_wc", "b_unfaithful", "ownmoney", "r_educ", "r_age", "cons"]
x5 = ["husb_age", "husb_educ", "husb_wc", "urban", "hhsize", "children", "rep_educ", "rep_age", "rep_wc", "b_unfaithful", "ownmoney", "agegap_cat", "educgap_cat", "cons"]

yeardummy = []
statedummy = []
religiondummy = []
husbdummy = []
repdummy = []
agedummy = []
educdummy = []
Xhusbrepdummy = []
for element in nfhs.columns:
    if element.startswith("state_"):
        statedummy.append(element)
    elif element.startswith("year_"):
        yeardummy.append(element)
    elif element.startswith("religion_"):
        religiondummy.append(element)
    elif element.startswith("husbcat_"):
        husbdummy.append(element)        
    elif element.startswith("repcat_"):
        repdummy.append(element)  
    elif element.startswith("agecat_"):
        agedummy.append(element)  
    elif element.startswith("educcat_"):
        educdummy.append(element)
        
educdummy.pop(0)        
educdummy.pop(0)
agedummy.pop(0)        
agedummy.pop(0)
husbdummy.pop(0)        
husbdummy.pop(0)        
repdummy.pop(0)        
repdummy.pop(0)
religiondummy.pop(0)
religiondummy.pop(0)
statedummy.pop(0)
yeardummy.pop(0)

for i in husbdummy:
    for j in repdummy:
        nfhs["Xhusb{}rep{}".format(i[-1],j[-1])] = nfhs[i]*nfhs[j]
        
for element in nfhs.columns:
    if element.startswith("Xhusb"):
        Xhusbrepdummy.append(element)
        
#Exog with controls
x1c = ["prohib", "literacy", "purban", "pcgdp", "unemp", "pcpolice", "pcpolice_exp", "pmale", "health", "educ", "cons"]

cons = []
for i in range(0, len(nfhs)):
    cons.append(1)
nfhs["cons"] = cons

cons = []
for i in range(0, len(crime)):
    cons.append(1)
crime["cons"] = cons

#Sets the endogenous variable
y = "husb_drink"

C:\Users\vboxuser\AppData\Local\Temp\ipykernel_6964\1182456989.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nfhs["Xhusb{}rep{}".format(i[-1],j[-1])] = nfhs[i]*nfhs[j]
C:\Users\vboxuser\AppData\Local\Temp\ipykernel_6964\1182456989.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nfhs["Xhusb{}rep{}".format(i[-1],j[-1])] = nfhs[i]*nfhs[j]
C:\Users\vboxuser\AppData\Local\Temp\ipykernel_6964\1182456989.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [5]:
x = x1c + yeardummy
while 0 < 1:
    try:
        results = mt.reg(nfhs, y, x,cluster='State',awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results = mt.reg(nfhs, y, x,cluster='State',awt_name="stwt")
        if len(varlist) == 0:
            break          
print(results)

Dependent variable:	husb_drink
N:			77842
R-squared:		0.0518
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  State
  No. of clusters:	  17
              coeff    se      t   p>t CI_low CI_high
prohib       -0.156 0.047 -3.323 0.004 -0.256  -0.056
literacy      0.001 0.003  0.333 0.743 -0.005   0.008
purban       -0.001 0.002 -0.749 0.465 -0.006   0.003
pcgdp         0.000 0.000  0.276 0.786 -0.000   0.000
unemp         0.001 0.012  0.116 0.909 -0.023   0.026
pcpolice      0.037 0.075  0.495 0.627 -0.121   0.195
pcpolice_exp -0.002 0.004 -0.427 0.675 -0.011   0.007
pmale        -0.769 0.523 -1.470 0.161 -1.878   0.340
health       -0.020 0.018 -1.115 0.281 -0.057   0.018
educ         -0.006 0.007 -0.806 0.432 -0.022   0.010
cons          0.833 0.320  2.606 0.019  0.155   1.510
year_1998    -0.026 0.053 -0.498 0.625 -0.138   0.086
year_1999    -0.024 0.050 -0.478 0.639 -0.130   0.082
year_2005    -0.108 0.042 -2.556 0.021 -0.198  -0.019



In [6]:
x = x2+statedummy+yeardummy+religiondummy
while 0 < 1:
    try:
        results2a = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results2a = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(results2a)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in results2a.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = results2a.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_drink"]-nfhs["dresid"]
x = x1c + yeardummy
while 0 < 1:
    try:
        results2 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results2 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(results2)

Dependent variable:	husb_drink
N:			98780
R-squared:		0.1344
Estimation method:	OLS
VCE method:		Standard (Homosk.)
            coeff    se       t   p>t CI_low CI_high
husb_age   -0.001 0.000  -8.156 0.000 -0.002  -0.001
husb_educ  -0.006 0.000 -28.472 0.000 -0.007  -0.006
husb_wc    -0.092 0.004 -25.232 0.000 -0.100  -0.085
urban       0.009 0.003   2.667 0.008  0.002   0.016
hhsize     -0.001 0.001  -1.594 0.111 -0.002   0.000
cons        0.607 0.009  67.002 0.000  0.589   0.624
state_1     0.211 0.013  16.018 0.000  0.185   0.237
state_2    -0.006 0.009  -0.676 0.499 -0.025   0.012
state_3    -0.054 0.009  -6.255 0.000 -0.071  -0.037
state_4     0.115 0.014   8.295 0.000  0.088   0.143
state_5    -0.042 0.010  -4.139 0.000 -0.062  -0.022
state_6    -0.306 0.009 -33.553 0.000 -0.323  -0.288
state_7    -0.014 0.012  -1.147 0.251 -0.038   0.010
state_8    -0.028 0.011  -2.595 0.009 -0.049  -0.007
state_9    -0.084 0.010  -8.152 0.000 -0.105  -0.064
state_10   -0.120 0.011 -11.046 0.00

In [7]:
x = x3+statedummy+yeardummy+religiondummy
while 0 < 1:
    try:
        results3a = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results3a = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(results3a)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in results3a.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = results3a.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_drink"]-nfhs["dresid"]
x = x1c + yeardummy
while 0 < 1:
    try:
        results3 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results3 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(results3)

Dependent variable:	husb_drink
N:			98755
R-squared:		0.1427
Estimation method:	OLS
VCE method:		Standard (Homosk.)
              coeff    se       t   p>t CI_low CI_high
husb_age     -0.000 0.001  -0.346 0.730 -0.003   0.002
husb_educ    -0.003 0.000 -13.233 0.000 -0.004  -0.003
husb_wc      -0.063 0.004 -16.586 0.000 -0.071  -0.056
urban         0.033 0.004   9.290 0.000  0.026   0.040
hhsize       -0.006 0.001  -7.901 0.000 -0.008  -0.005
children      0.010 0.001   7.943 0.000  0.007   0.012
rep_educ     -0.012 0.000 -27.027 0.000 -0.013  -0.011
rep_age      -0.002 0.001  -1.594 0.111 -0.005   0.001
rep_wc        0.028 0.007   4.214 0.000  0.015   0.041
r_educ        0.010 0.001   7.157 0.000  0.007   0.013
r_age         0.059 0.058   1.009 0.313 -0.056   0.174
b_unfaithful  0.011 0.003   3.434 0.001  0.005   0.017
ownmoney      0.024 0.003   8.023 0.000  0.018   0.030
cons          0.582 0.050  11.629 0.000  0.484   0.680
state_1       0.215 0.013  16.273 0.000  0.189   0.241
stat

In [8]:
x = x4+statedummy+yeardummy+religiondummy+husbdummy+repdummy+Xhusbrepdummy
while 0 < 1:
    try:
        results4a = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results4a = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(results4a)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in results4a.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = results4a.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_drink"]-nfhs["dresid"]

x = x1c + yeardummy
while 0 < 1:
    try:
        results4 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        result4 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(results4)

Dependent variable:	husb_drink
N:			98755
R-squared:		0.1445
Estimation method:	OLS
VCE method:		Standard (Homosk.)
              coeff    se       t   p>t CI_low CI_high
husb_age     -0.005 0.002  -2.105 0.035 -0.010  -0.000
husb_educ    -0.003 0.000 -13.184 0.000 -0.004  -0.003
husb_wc      -0.064 0.004 -16.734 0.000 -0.071  -0.056
urban         0.032 0.004   9.067 0.000  0.025   0.039
hhsize       -0.007 0.001  -8.269 0.000 -0.008  -0.005
children      0.007 0.001   6.028 0.000  0.005   0.010
rep_educ     -0.012 0.000 -27.628 0.000 -0.013  -0.012
rep_age       0.003 0.003   1.164 0.244 -0.002   0.008
rep_wc        0.026 0.007   3.960 0.000  0.013   0.039
b_unfaithful  0.011 0.003   3.388 0.001  0.005   0.017
ownmoney      0.024 0.003   7.846 0.000  0.018   0.030
r_educ        0.010 0.001   7.431 0.000  0.008   0.013
r_age        -0.144 0.108  -1.331 0.183 -0.356   0.068
cons          0.680 0.107   6.346 0.000  0.470   0.890
state_1       0.216 0.013  16.349 0.000  0.190   0.242
stat

In [9]:
x = x5+statedummy+yeardummy+religiondummy+agedummy+educdummy
while 0 < 1:
    try:
        results5a = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        results5a = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(results5a)
            
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in results5a.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = results5a.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_drink"]-nfhs["dresid"]

x = x1c + yeardummy
while 0 < 1:
    try:
        results5 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        result5 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(results5)

Dependent variable:	husb_drink
N:			98755
R-squared:		0.1454
Estimation method:	OLS
VCE method:		Standard (Homosk.)
              coeff    se       t   p>t CI_low CI_high
husb_age     -0.002 0.001  -1.605 0.108 -0.004   0.000
husb_educ    -0.001 0.000  -3.403 0.001 -0.002  -0.001
husb_wc      -0.052 0.004 -13.548 0.000 -0.060  -0.045
urban         0.036 0.004   9.972 0.000  0.029   0.042
hhsize       -0.006 0.001  -7.509 0.000 -0.008  -0.005
children      0.009 0.001   7.473 0.000  0.007   0.011
rep_educ     -0.016 0.001 -27.552 0.000 -0.017  -0.015
rep_age      -0.001 0.001  -0.727 0.467 -0.003   0.001
rep_wc        0.025 0.007   3.793 0.000  0.012   0.038
b_unfaithful  0.011 0.003   3.347 0.001  0.004   0.017
ownmoney      0.025 0.003   8.416 0.000  0.020   0.031
agegap_cat    0.001 0.005   0.277 0.782 -0.008   0.011
educgap_cat   0.011 0.024   0.452 0.651 -0.037   0.059
cons          0.647 0.010  64.948 0.000  0.627   0.666
state_1       0.213 0.013  16.129 0.000  0.187   0.238
stat

In [10]:
y = ["husb_beat"]

In [11]:
x = x1c + yeardummy
while 0 < 1:
    try:
        bresults = mt.reg(nfhs, y, x,cluster='State',awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults = mt.reg(nfhs, y, x,cluster='State',awt_name="stwt")
        if len(varlist) == 0:
            break          
print(bresults)

Dependent variable:	['husb_beat']
N:			77848
R-squared:		0.0423
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  State
  No. of clusters:	  17
              coeff    se      t   p>t CI_low CI_high
prohib       -0.084 0.038 -2.208 0.042 -0.165  -0.003
literacy     -0.005 0.001 -3.345 0.004 -0.008  -0.002
purban        0.003 0.002  1.927 0.072 -0.000   0.007
pcgdp         0.000 0.000  0.101 0.921 -0.000   0.000
unemp         0.003 0.008  0.415 0.684 -0.014   0.020
pcpolice     -0.047 0.046 -1.020 0.323 -0.144   0.050
pcpolice_exp  0.002 0.003  0.784 0.444 -0.004   0.008
pmale        -0.461 0.346 -1.333 0.201 -1.193   0.272
health       -0.014 0.011 -1.303 0.211 -0.036   0.009
educ          0.005 0.004  1.146 0.269 -0.004   0.014
cons          0.597 0.136  4.390 0.000  0.309   0.885
year_1998     0.031 0.036  0.858 0.404 -0.045   0.106
year_1999     0.047 0.039  1.203 0.246 -0.036   0.130
year_2005     0.029 0.032  0.909 0.377 -0.039   0.097



In [12]:
x = x2+statedummy+yeardummy+religiondummy
while 0 < 1:
    try:
        bresults2b = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults2b = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(bresults2b)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in bresults2b.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = bresults2b.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_beat"]-nfhs["dresid"]
x = x1c + yeardummy
while 0 < 1:
    try:
        bresults2 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults2 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(bresults2)

Dependent variable:	['husb_beat']
N:			98785
R-squared:		0.0685
Estimation method:	OLS
VCE method:		Standard (Homosk.)
            coeff    se       t   p>t CI_low CI_high
husb_age   -0.000 0.000  -1.162 0.245 -0.000   0.000
husb_educ  -0.004 0.000 -22.541 0.000 -0.004  -0.004
husb_wc    -0.041 0.003 -13.623 0.000 -0.047  -0.035
urban      -0.027 0.003  -9.416 0.000 -0.032  -0.021
hhsize      0.001 0.001   1.732 0.083 -0.000   0.002
cons        0.218 0.007  29.187 0.000  0.204   0.233
state_1    -0.031 0.011  -2.854 0.004 -0.053  -0.010
state_2    -0.054 0.008  -6.994 0.000 -0.070  -0.039
state_3     0.058 0.007   8.151 0.000  0.044   0.072
state_4    -0.010 0.012  -0.886 0.375 -0.033   0.012
state_5    -0.076 0.008  -8.927 0.000 -0.092  -0.059
state_6    -0.099 0.008 -13.184 0.000 -0.114  -0.084
state_7    -0.076 0.010  -7.428 0.000 -0.096  -0.056
state_8    -0.158 0.009 -17.977 0.000 -0.176  -0.141
state_9    -0.052 0.009  -6.040 0.000 -0.068  -0.035
state_10   -0.095 0.009 -10.630 0

In [13]:
x = x3+statedummy+yeardummy+religiondummy
while 0 < 1:
    try:
        bresults3b = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults3b = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(bresults3b)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in bresults3b.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = bresults3b.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_beat"]-nfhs["dresid"]
x = x1c + yeardummy
while 0 < 1:
    try:
        bresults3 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults3 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(bresults3)

Dependent variable:	['husb_beat']
N:			98760
R-squared:		0.0809
Estimation method:	OLS
VCE method:		Standard (Homosk.)
              coeff    se       t   p>t CI_low CI_high
husb_age      0.002 0.001   1.668 0.095 -0.000   0.004
husb_educ    -0.001 0.000  -7.106 0.000 -0.002  -0.001
husb_wc      -0.014 0.003  -4.602 0.000 -0.021  -0.008
urban        -0.002 0.003  -0.652 0.514 -0.008   0.004
hhsize       -0.005 0.001  -7.231 0.000 -0.006  -0.004
children      0.011 0.001  11.131 0.000  0.009   0.013
rep_educ     -0.010 0.000 -27.201 0.000 -0.011  -0.009
rep_age      -0.003 0.001  -2.863 0.004 -0.006  -0.001
rep_wc        0.032 0.005   5.765 0.000  0.021   0.042
r_educ        0.007 0.001   6.534 0.000  0.005   0.010
r_age         0.082 0.048   1.705 0.088 -0.012   0.177
b_unfaithful  0.040 0.003  15.025 0.000  0.034   0.045
ownmoney      0.003 0.002   1.082 0.279 -0.002   0.008
cons          0.166 0.041   4.016 0.000  0.085   0.247
state_1      -0.018 0.011  -1.650 0.099 -0.039   0.003
s

In [14]:
x = x4+statedummy+yeardummy+religiondummy+husbdummy+repdummy+Xhusbrepdummy
while 0 < 1:
    try:
        bresults4b = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults4b = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(bresults4b)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in bresults4b.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = bresults4b.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_beat"]-nfhs["dresid"]
x = x1c + yeardummy
while 0 < 1:
    try:
        bresults4 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults4 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(bresults4)

Dependent variable:	['husb_beat']
N:			98760
R-squared:		0.0822
Estimation method:	OLS
VCE method:		Standard (Homosk.)
              coeff    se       t   p>t CI_low CI_high
husb_age      0.001 0.002   0.329 0.742 -0.003   0.004
husb_educ    -0.001 0.000  -7.082 0.000 -0.002  -0.001
husb_wc      -0.015 0.003  -4.724 0.000 -0.021  -0.009
urban        -0.002 0.003  -0.822 0.411 -0.008   0.003
hhsize       -0.005 0.001  -7.412 0.000 -0.006  -0.004
children      0.009 0.001   9.441 0.000  0.008   0.011
rep_educ     -0.010 0.000 -27.622 0.000 -0.011  -0.010
rep_age      -0.003 0.002  -1.186 0.236 -0.007   0.002
rep_wc        0.031 0.005   5.641 0.000  0.020   0.042
b_unfaithful  0.040 0.003  15.027 0.000  0.034   0.045
ownmoney      0.002 0.002   0.935 0.350 -0.003   0.007
r_educ        0.008 0.001   6.778 0.000  0.005   0.010
r_age         0.066 0.089   0.736 0.462 -0.109   0.241
cons          0.033 0.088   0.375 0.708 -0.140   0.206
state_1      -0.017 0.011  -1.549 0.121 -0.038   0.004
s

In [15]:
x = x5+statedummy+yeardummy+religiondummy+agedummy+educdummy
while 0 < 1:
    try:
        bresults5b = mt.reg(nfhs, y, x,awt_name="stwt",check_colinear=True)
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults5b = mt.reg(nfhs, y, x,awt_name="stwt")
        if len(varlist) == 0:
            break          

print(bresults5b)
    
dresid = []
for i in range(0, len(nfhs)):
    dresid.append(0)
nfhs["dresid"] = dresid
for i in bresults5b.x_name:
        if i.startswith("state_"):
            continue
        elif i.startswith("year_"):
            continue
        else:
            nfhs["dresid"] = bresults5b.beta[i]*nfhs[i]+nfhs["dresid"]
nfhs["dresid"] = nfhs["husb_beat"]-nfhs["dresid"]
x = x1c + yeardummy
while 0 < 1:
    try:
        bresults5 = mt.reg(nfhs, "dresid", x,awt_name="stwt",check_colinear=True,cluster='State')
        break
    except ValueError as err:
        errstring = err.args[0]
        varlist = errstring.split("\n")
        varlist.pop(0)
        for i in range(0,len(varlist)):
            x.remove(varlist[i])
        bresults5 = mt.reg(nfhs, "dresid", x,awt_name="stwt",cluster='State')
        if len(varlist) == 0:
            break
print(bresults5)

Dependent variable:	['husb_beat']
N:			98760
R-squared:		0.0828
Estimation method:	OLS
VCE method:		Standard (Homosk.)
              coeff    se       t   p>t CI_low CI_high
husb_age      0.002 0.001   2.169 0.030  0.000   0.004
husb_educ     0.000 0.000   0.776 0.438 -0.000   0.001
husb_wc      -0.007 0.003  -2.246 0.025 -0.013  -0.001
urban        -0.000 0.003  -0.134 0.893 -0.006   0.005
hhsize       -0.005 0.001  -6.895 0.000 -0.006  -0.003
children      0.011 0.001  10.759 0.000  0.009   0.013
rep_educ     -0.013 0.000 -27.144 0.000 -0.014  -0.012
rep_age      -0.003 0.001  -3.728 0.000 -0.005  -0.002
rep_wc        0.030 0.005   5.433 0.000  0.019   0.040
b_unfaithful  0.040 0.003  15.002 0.000  0.034   0.045
ownmoney      0.003 0.002   1.360 0.174 -0.001   0.008
agegap_cat   -0.005 0.004  -1.154 0.249 -0.012   0.003
educgap_cat   0.051 0.020   2.529 0.011  0.011   0.090
cons          0.239 0.008  29.129 0.000  0.223   0.256
state_1      -0.019 0.011  -1.774 0.076 -0.041   0.002
s

In [16]:
print("The prohib coefficients were {}, {}, {}, {}, {} for panel A table 1".format(results.beta["prohib"],results2.beta["prohib"],results3.beta["prohib"],results4.beta["prohib"],results5.beta["prohib"]))
print("The prohib standard errors were {}, {}, {}, {}, {} for panel A table 1".format(results.se["prohib"],results2.se["prohib"],results3.se["prohib"],results4.se["prohib"],results5.se["prohib"]))
print()
print("The prohib coefficients were {}, {}, {}, {}, {} for panel B table 1".format(bresults.beta["prohib"],bresults2.beta["prohib"],bresults3.beta["prohib"],bresults4.beta["prohib"],bresults5.beta["prohib"]))
print("The prohib standard errors were {}, {}, {}, {}, {} for panel B table 1".format(bresults.se["prohib"],bresults2.se["prohib"],bresults3.se["prohib"],bresults4.se["prohib"],bresults5.se["prohib"]))

The prohib coefficients were -0.15602765212327163, -0.13456155019838123, -0.13135300513670445, -0.13096774282286872, -0.13491169048877694 for panel A table 1
The prohib standard errors were 0.046956036926873286, 0.0357556123028628, 0.03482469680475576, 0.03497811729181679, 0.03474061688029193 for panel A table 1

The prohib coefficients were -0.08398834654537779, -0.08267499953939722, -0.07902503212764427, -0.07852038123859387, -0.08170194015548045 for panel B table 1
The prohib standard errors were 0.03804057511900899, 0.03824900436973027, 0.038422660113671, 0.03849673462512065, 0.03814878823662197 for panel B table 1


In [17]:
crime.dropna(axis=0,inplace=True)
crime.reset_index(inplace=True)
statedummy = []
yeardummy = []
for element in crime.columns:
    if element.startswith("state_"):
        statedummy.append(element)
    elif element.startswith("year_"):
        yeardummy.append(element)
statedummy.pop(0)
yeardummy.pop(0)

'year_1980'

In [18]:
from sklearn.utils import resample
boot = resample(crime, replace=True, n_samples=1000, random_state=10101)

In [19]:
y = ["women_indexnf", "r_cruelty", "r_molestation", "r_sexualharassment", "r_rape", "women_index_f", "r_suicides_f", "r_dowrydeath", "r_fire_f", "women_index"]
coef = []
std = []
for r in y:
    x = ["prohib", "literacy", "urban", "pcgdp", "unemp", "pcpolice", "cons"]+statedummy+yeardummy
    while 0 < 1:
        try:
            model = mt.reg(boot, r, x, awt_name="pop",check_colinear=True,cluster='State')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            model = mt.reg(boot, r, x, awt_name="pop",cluster='State')
            if len(varlist) == 0:
                break
    print(model)
    coef.append(model.beta["prohib"])
    std.append(model.se["prohib"])

Dependent variable:	women_indexnf
N:			1000
R-squared:		0.9024
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  State
  No. of clusters:	  14
              coeff       se      t   p>t     CI_low  CI_high
prohib     -274.806   94.006 -2.923 0.012   -477.893  -71.719
literacy     19.596   33.954  0.577 0.574    -53.756   92.949
urban        67.117   16.767  4.003 0.002     30.893  103.341
pcgdp         0.024    0.012  2.071 0.059     -0.001    0.050
unemp       103.833   47.325  2.194 0.047      1.593  206.072
pcpolice    532.329  312.012  1.706 0.112   -141.733 1206.390
cons      -4186.670 2773.780 -1.509 0.155 -10179.056 1805.717
state_1    1331.490  351.831  3.784 0.002    571.406 2091.574
state_4    1421.249  891.365  1.594 0.135   -504.429 3346.927
state_10     -2.572  258.744 -0.010 0.992   -561.554  556.410
state_13    317.974  194.314  1.636 0.126   -101.816  737.765
state_16   -172.909  157.789 -1.096 0.293   -513.791  167.972
state_17   -710.721  852.216 -0.834

In [20]:
i = 0
print("The prohib coefficients were {}, {}, {}, {}, {} for panel A table 2".format(coef[i],coef[i+1],coef[i+2],coef[i+3],coef[i+4]))
print("The prohib standard errors were {}, {}, {}, {}, {} for panel A table 2".format(std[i],std[i+1],std[i+2],std[i+3],std[i+4]))
print()
print("The prohib coefficients were {}, {}, {}, {}, {} for panel B table 2".format(coef[i+5],coef[i+6],coef[i+7],coef[i+8],coef[i+9]))
print("The prohib standard errors were {}, {}, {}, {}, {} for panel B table 2".format(std[i+5],std[i+6],std[i+7],std[i+8],std[i+9]))

The prohib coefficients were -274.8060817391415, -195.1218879562166, -1.6418332353914593, -72.42180328570385, -5.620552139924712 for panel A table 2
The prohib standard errors were 94.00590405699772, 63.35648434170359, 29.686720167625634, 35.08535414553405, 4.626136211312714 for panel A table 2

The prohib coefficients were -58.00213340944356, -45.85189071361674, 9.477226869153583, -21.627475873978256, -332.8082061097084 for panel B table 2
The prohib standard errors were 23.517788959271552, 11.750175967682987, 4.414848604015501, 16.414044309091476, 111.48254150829425 for panel B table 2
